In [4]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from pathlib import Path

In [5]:
def process_all_pdfs(pdf_directory):
    """Process all pdf files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    # Find all PDF files recursively

    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\n Processing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages") 

        except Exception as e:
            print(f" Error : {e}") 

        print(f"\n Total documents loaded : {len(all_documents)}")    
        return all_documents;

all_pdf_documents = process_all_pdfs("data\\pdf\\")      


Found 1 PDF files to process

 Processing: GenericAITest.pdf
 Loaded 3 pages

 Total documents loaded : 3


In [6]:
#### Text splitting get into chunks

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_spliter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ",""]
    )
    split_docs = text_spliter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    #show example of a chunks

    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs    

In [7]:
chunks = split_documents(all_pdf_documents)
print(chunks)

Split 3 documents into 17 chunks

Example chunk:
Content: https://crsreports.congress.gov 
 
Updated April 2, 2025
Generative Artificial Intelligence: Overview, Issues, and 
Considerations for Congress 
Generative artificial intelligence (GenAI) refers to AI...
Metadata: {'producer': 'iText® 7.1.2 ©2000-2018 iText Group NV (AGPL-version); modified using iText® 7.1.2 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-04-04T16:06:39-04:00', 'moddate': '2025-04-04T16:06:43-04:00', 'source': 'data\\pdf\\GenericAITest.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'GenericAITest.pdf', 'file_type': 'pdf'}
[Document(metadata={'producer': 'iText® 7.1.2 ©2000-2018 iText Group NV (AGPL-version); modified using iText® 7.1.2 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-04-04T16:06:39-04:00', 'moddate': '2025-04-04T16:06:43-04:00', 'source': 'data\\pdf\\GenericAITest.pdf', 'total_pages': 3, 'page': 0, 'p

In [8]:
# Embedding and vector db
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any , Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer"""
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        Args : model_name : Hugging face model for sentence embeddings
        ll-MiniLM-L6-v2  - this model is responsible for converting text into vectors
        """

        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer Model"""    
        try:
            print(f"Loading embedding model : {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension : {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error while loading model {self.model_name}: {e}") 
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
       """Generate embeddings for a list of texts
       Args : list of text strings to embed
       Returns :
       numpy array of embeddings with shape (len(texts), embedding_dim)
       """
       if not self.model:
           raise ValueError("Model not loaded")
       
       print (f"Generating embeddings for {len(texts)} texts")
       embeddings = self.model.encode(texts, show_progress_bar=True)
       print(f"Generated embeddings with shape : {embeddings.shape}")
       return embeddings
    

embedding_manager = EmbeddingManager()
print(embedding_manager)



Loading embedding model : all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension : 384


In [10]:
import os
### Vector Store
class VectorStore:
    """Manges document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "all_pdf_documents", persist_directory: str = "data\\vector_store"):
        """Initialzie the vector store
        Args: 
        collection_name: Name of the ChromaDB collection
        persist_directory: Directory to persist the vector store

        """

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initilize ChromaDB client and collection"""
        try:
            # create persistent ChromaDB Client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection

            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata={"description" : "PDF document embeddings for RAG"}
            )    
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing docuemnts in collection : {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vecotr store : {e}")
            raise    


    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """       
            Add documents and their embeddings to the vectore store.
            Args : List of Langchain documents
            embeddings : Corresponding embeddings for the documents. 
        """

        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} docuemnts to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embedding_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            #Document Content
            documents_text.append(doc.page_content)

            #Embedding
            embedding_list.append(embedding.tolist())
            

            # Add to collection
            try:
                self.collection.add(
                    ids=ids,
                    embeddings=embedding_list,
                    metadatas=metadatas,
                    documents=documents_text
                )
                print(f"Successfully added {len(documents)} documents to vector store")
                print(f"Total docuemnts in collection : {self.collection.count}")
            except Exception as e:
                print(f"Error adding documents to vector store: {e}")    
                raise

vectorStore = VectorStore()
vectorStore


Vector store initialized. Collection: all_pdf_documents
Existing docuemnts in collection : 17


In [11]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generte the embeddings
embeddings = embedding_manager.generate_embeddings(texts)


# Store in the vector database
vectorStore.add_documents(chunks, embeddings)

Generating embeddings for 17 texts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape : (17, 384)
Adding 17 docuemnts to vector store...
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully added 17 documents to vector store
Total docuemnts in collection : <bound method Collection.count of Collection(name=all_pdf_documents)>
Successfully adde

In [12]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorStore,embedding_manager)

In [13]:
print(rag_retriever)

In [14]:
rag_retriever.retrieve("what is Generative Artificail intelligence refer ?")

Retrieving documents for query: 'what is Generative Artificail intelligence refer ?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape : (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_1ae960f8_0',
  'content': 'https://crsreports.congress.gov \n \nUpdated April 2, 2025\nGenerative Artificial Intelligence: Overview, Issues, and \nConsiderations for Congress \nGenerative artificial intelligence (GenAI) refers to AI \nmodels, in particular those that use machine learning (ML) \nand are trained on large volumes of data, that are able to \ngenerate new content. In contrast, other AI models may \nhave a primary goal of classifying data, such as facial \nrecognition image data, or making decisions, such as those \nused in automated vehicles. GenAI, when prompted (often \nby a user inputting text), can create various outputs, \nincluding text, images, videos, computer code, or music. \nThe public release of many GenAI tools, and the race by \ncompanies to develop ever-more powerful AI models, have \ngenerated widespread discussion of their capabilities, \npotential concerns with their use, and debates about their \ngovernance and regulation. This CRS In Focus d

In [15]:
rag_retriever.retrieve("What are the capabilities of AI ?")

Retrieving documents for query: 'What are the capabilities of AI ?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape : (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_ea63c386_1',
  'content': 'generated widespread discussion of their capabilities, \npotential concerns with their use, and debates about their \ngovernance and regulation. This CRS In Focus describes \nthe development and uses of GenAI, concerns raised by the \nuse of GenAI tools, and considerations for Congress.  \nBackground \nAI can generally be thought of as computerized systems \nthat work and react in ways commonly considered to \nrequire human intelligence, such as learning, solving \nproblems, and achieving goals under uncertain and varying \nconditions, with varying levels of autonomy. AI can \nencompass a range of technologies, methodologies, and \napplication areas, such as natural language processing, \nrobotics, and facial recognition. \nThe AI technologies underpinning many GenAI tools are \nthe result of decades of research. For example, recurrent \nneural networks (RNNs), a type of ML loosely modeled \nafter the human brain that detects patterns in sequenti

In [ ]:
#Simple RAG Pipeline with Open AI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

load_dotenv()

# Initialize the Open AI LLM 
model = init_chat_model(
    "gpt-4.1",
    temperature=0.5,
    timeout=30,
    max_tokens=1000)
response = model.invoke("What is Generative AI  ?")
print(response)

# Simple RAG Function
def rag_simple(query, retriever, model, top_k=3):
    ## retrieve the context 
    results=retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context fount to answer to the question. "
    
    ## Generate the answer using open ai 
    prompt=f""" Use the following context to answer the question concisely. 
        Context:
        {context}
        Question: {query}
        Answer: """
    print(f"Prompt : {prompt}")
    response=model.invoke([prompt.format(context=context, query=query)])
    return response.content

content='**Generative AI** is a type of artificial intelligence that can create new content, such as text, images, music, code, and more. Unlike traditional AI, which typically analyzes data and makes predictions or classifications, generative AI can produce original material that resembles the examples it was trained on.\n\n### Key Points:\n- **"Generative"** means it can generate or create things.\n- It uses advanced machine learning models, especially neural networks like **GPT** (for text), **DALL-E** or **Stable Diffusion** (for images), and others.\n- It learns patterns from large datasets and uses that knowledge to create new, similar content.\n\n### Examples:\n- **Text:** Chatbots like ChatGPT that write essays, answer questions, or generate stories.\n- **Images:** AI tools that create artwork or realistic photos from text prompts.\n- **Music:** AI systems that compose original songs.\n- **Code:** AI that writes programming code based on natural language instructions.\n\n### Ho

In [20]:
answer = rag_simple("What is the use of Generative AI ? ", rag_retriever, model)
print(answer)

Retrieving documents for query: 'What is the use of Generative AI ? '
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape : (1, 384)
Retrieved 3 documents (after filtering)
Prompt :  Use the following context to answer the question concisely. 
        Context:
        https://crsreports.congress.gov 
 
Updated April 2, 2025
Generative Artificial Intelligence: Overview, Issues, and 
Considerations for Congress 
Generative artificial intelligence (GenAI) refers to AI 
models, in particular those that use machine learning (ML) 
and are trained on large volumes of data, that are able to 
generate new content. In contrast, other AI models may 
have a primary goal of classifying data, such as facial 
recognition image data, or making decisions, such as those 
used in automated vehicles. GenAI, when prompted (often 
by a user inputting text), can create various outputs, 
including text, images, videos, computer code, or music. 
The public release of many GenAI tools, and the race by 
companies to develop ever-more powerful AI models, have 
generated widespread discussion of their 